# Pandas support

It is convenient to use the Pandas package when dealing with numerical data, so Pint provides PintArray. A PintArray is a Pandas Extension Array, which allows Pandas to recognise the Quantity and store it in Pandas DataFrames and Series.

## Basic example

This example will show the simplist way to use pandas with pint and the underlying objects. It's slightly fiddly as you are not reading from a file. A more normal use case is given in Reading a csv.

First some imports

In [8]:
import pandas as pd
import pint
import pint_pandas
import numpy as np

In [9]:
ureg = pint.UnitRegistry(autoconvert_offset_to_baseunit=True)

In [11]:
pint_pandas.PintType.ureg = ureg

In [15]:
# ureg = UnitRegistry(auto_reduce_dimensions=False)
# ureg.define('bit = [information]')
ureg.default_format = '~H'
Q_ = ureg.Quantity

In [16]:
start_date = '2010-01-01'
end_date = '2030-12-01'
date_range = pd.date_range(start_date, end_date, freq='MS')
samples = 3

In [17]:
iterables = [date_range, range(samples)]
index_names = ['time', 'samples']
_multi_index = pd.MultiIndex.from_product(iterables, names=index_names)

In [18]:
pdf = pd.DataFrame(index=_multi_index, dtype='pint[W]')
sdf = pd.DataFrame(index=_multi_index, dtype='pint[W]')
pdf['power'] = np.full((len(date_range), samples), 5.3).ravel()
sdf['time'] = np.full((len(date_range), samples), 5).ravel()

In [19]:
power = pd.Series(np.full((len(date_range), samples), 5.5).ravel(), index=_multi_index,  dtype='pint[W]')
time = pd.Series(np.full((len(date_range), samples), 5).ravel(), index=_multi_index,  dtype='pint[s]')

/home/james/envs/eam-env-2/lib/python3.8/site-packages/pint_pandas/pint_array.py:194: RuntimeWarning: pint-pandas does not support magnitudes of <class 'numpy.int64'>. Converting magnitudes to float.
  warnings.warn(


In [20]:
pdf['time'] = sdf['time']

## Evaluate operations with multi-level column indexes

In [21]:
d = {}
d['UK'] = pdf
UK = pd.concat(d, axis=1)
UK.head(5)

UK     
                   power time
time       samples           
2010-01-01 0         5.3    5
           1         5.3    5
           2         5.3    5
2010-02-01 0         5.3    5
           1         5.3    5

In [22]:
d = {}
d['US'] = pdf
US = pd.concat(d, axis=1)
US.head(5)

US     
                   power time
time       samples           
2010-01-01 0         5.3    5
           1         5.3    5
           2         5.3    5
2010-02-01 0         5.3    5
           1         5.3    5

In [23]:
US.mul(UK, axis=1, level=1)

TypeError: Join on level between two MultiIndex objects is ambiguous

In [24]:
sdf.pint.units

AttributeError: 'PintDataFrameAccessor' object has no attribute 'units'

In [25]:
df = pd.DataFrame(index=_multi_index)

In [26]:
df['power'] = power
df['time'] = time

In [27]:
isinstance(df, pd.DataFrame)

True

In [28]:
df.pint.dequantify().head()


power time
unit                   W    s
time       samples           
2010-01-01 0         5.5  5.0
           1         5.5  5.0
           2         5.5  5.0
2010-02-01 0         5.5  5.0
           1         5.5  5.0

In [ ]:
df= df.pint.dequantify()
df.columns = df.columns.droplevel(1)
df.head()

In [ ]:
df.mean(level='time')

In [ ]:
df['power'].dtype

In [ ]:
df.pint.dequantify()['power']['watt'].dtype

In [ ]:
x = power * time

In [ ]:
x.pint.to('Wh').iloc[0].m

In [ ]:
x.pint.to('Wh')

In [ ]:
'{:.02f~H}'.format(Q_(power.pint.m.mean(),power.pint.units))

In [ ]:
Q()

In [ ]:
df['energy'].pint.to('J').head()

In [ ]:
df['power'].pint.units

In [ ]:
df['energy'].pint.to('J').pint.m.head()

In [ ]:
df['energy'].pint.to('Wh').head()

In [ ]:
df['energy'].pint.to('Wh').pint.m.mean(level='time')

Next, we create a DataFrame with PintArrays as columns.

In [ ]:
df = pd.DataFrame({
    "torque": pd.Series([1, 2, 2, 3], dtype="pint[lbf ft]"),
    "angular_velocity": pd.Series([1, 2, 2, 3], dtype="pint[rpm]"),
})
df

Operations with columns are units aware so behave as we would intuitively expect.

In [ ]:
df['power'] = df['torque'] * df['angular_velocity']
df

In [ ]:
{v[0]:v[1] for v in df.pint.dequantify().columns.values}

We can see the columns' units in the dtypes attribute

In [ ]:
df.dtypes

Each column can be accessed as a Pandas Series

In [ ]:
df.power

Which contains a PintArray

In [ ]:
df.power.values

The PintArray contains a Quantity

In [ ]:
df.power.values.quantity

Pandas Series accessors are provided for most Quantity properties and methods, which will convert the result to a Series where possible.

In [ ]:
df.power.pint.units

In [ ]:
df.power.pint.to("kW").values

## Reading from csv

Reading from files is the far more standard way to use pandas. To facilitate this, DataFrame accessors are provided to make it easy to get to PintArrays. 

In [ ]:
import pandas as pd 
import pint
import io

Here's the contents of the csv file.

In [ ]:
test_data = '''speed,mech power,torque,rail pressure,fuel flow rate,fluid power
rpm,kW,N m,bar,l/min,kW
1000.0,,10.0,1000.0,10.0,
1100.0,,10.0,100000000.0,10.0,
1200.0,,10.0,1000.0,10.0,
1200.0,,10.0,1000.0,10.0,'''

Let's read that into a DataFrame.
Here io.StringIO is used in place of reading a file from disk, whereas a csv file path would typically be used and is shown commented.

In [ ]:
df = pd.read_csv(io.StringIO(test_data),header=[0,1])
# df = pd.read_csv("/path/to/test_data.csv",header=[0,1])
df

Then use the DataFrame's pint accessor's quantify method to convert the columns from `np.ndarray`s to PintArrays, with units from the bottom column level.

In [ ]:
df.dtypes

In [ ]:
df_ = df.pint.quantify(level=-1)
df_

As previously, operations between DataFrame columns are unit aware

In [ ]:
df_.speed*df_.torque

In [ ]:
df_

In [ ]:
df_['mech power'] = df_.speed*df_.torque
df_['fluid power'] = df_['fuel flow rate'] * df_['rail pressure']
df_

The DataFrame's `pint.dequantify` method then allows us to retrieve the units information as a header row once again.

In [ ]:
df_.pint.dequantify()

This allows for some rather powerful abilities. For example, to change single column units

In [ ]:
df_['fluid power'] = df_['fluid power'].pint.to("kW")
df_['mech power'] = df_['mech power'].pint.to("kW")
df_.pint.dequantify()

The units are harder to read than they need be, so lets change pints default format for displaying units.

In [ ]:
pint.PintType.ureg.default_format = "~P"
df_.pint.dequantify()

or the entire table's units

In [ ]:
df_.pint.to_base_units().pint.dequantify()

## Advanced example
This example shows alternative ways to use pint with pandas and other features.

Start with the same imports.

In [ ]:
import pandas as pd 
import pint

We'll be use a shorthand for PintArray

In [ ]:
PA_ = pint.PintArray

And set up a unit registry and quantity shorthand.

In [ ]:
ureg=pint.UnitRegistry()
Q_=ureg.Quantity

Operations between PintArrays of different unit registry will not work. We can change the unit registry that will be used in creating new PintArrays to prevent this issue.

In [ ]:
pint.PintType.ureg = ureg

These are the possible ways to create a PintArray.

Note that pint[unit] must be used for the Series constuctor, whereas the PintArray constructor allows the unit string or object.

In [ ]:
df = pd.DataFrame({
        "length" : pd.Series([1,2], dtype="pint[m]"),
        "width" : PA_([2,3], dtype="pint[m]"),
        "distance" : PA_([2,3], dtype="m"),
        "height" : PA_([2,3], dtype=ureg.m),
        "depth" : PA_.from_1darray_quantity(Q_([2,3],ureg.m)),
    })
df

In [ ]:
df.length.values.units

In [ ]:
text = 'test'

In [ ]:
for i in range(0,10):
    print(str(i) + text)

In [ ]:
a

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plot

In [ ]:
plot.plot([0,1,2], [1,1,1])